In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
#Get modules
#indicators
from Incision import *
from Headcuts import *
from Erosion import *
from Invasives import *
from Bioassessment import *
from Conifer import *
from VegVigor import *
from AOP import *
from Habitat import *
from Ditches import *
from FormatStagingTables import *
from SEZScores import *



{'Angora Creek - tributary': 519, 'Angora Creek - upper': 520, 'Angora meadows - 1': 87, 'Angora meadows - 2': 90, 'Angora meadows - 3': 91, 'Angora meadows - 4': 143, 'Angora meadows - 5': 144, 'Angora meadows - 6': 142, 'Angora meadows - 7': 89, 'Angora meadows - 8': 88, 'Angora meadows - 9': 92, 'Angora meadows tributary - 1': 217, 'Angora meadows tributary - 2': 99, 'Angora meadows tributary - 3': 97, 'Angora meadows tributary - 4': 94, 'Angora meadows tributary - 5': 214, 'Angora meadows tributary - 6': 146, 'Angora meadows tributary - 7': 93, 'Angora meadows tributary - 8': 95, 'Angora meadows tributary - 9': 96, 'Angora tributary': 446, 'Antone meadows': 187, 'Baldwin marsh - 1': 160, 'Benwood meadows - 1': 129, 'Benwood meadows - 2': 131, 'Big Meadow - 1': 47, 'Big Meadow - 2': 48, 'Big Meadow - 3': 38, 'Big Meadow - 4': 39, 'Big Meadow - 5': 37, 'Big Meadow - 6': 40, 'Big meadow - 7': 126, 'Big Meadow Creek - lower': 521, 'Big Meadow Creek - upper': 491, 'Big Meadow Creek - up

# Part 1: Process Indicators Post To Staging Tables

## Process when new Data is Available = External Data Sources

In [ ]:
#CONIFER#
#!!!No Need to update if there is no new information
#Current code grabs old data which is already in staging table
from Conifer import *
df=get_conifersez_data_sql()
readydf=process_conifer(df)
post_conifer_data(df, draft=True)

In [ ]:
#AOP#
#!!! No need to update if there is no new information- 
# code isn't complete for new data- do manually for now until we set up a script for sde.aquaticorganismpassage_usfs
#Current code grabs old data which is already in staging table
from AOP import *
#Get data from SEZ_Assessment Unit Table in sde (this indicator doesn't get updated much)
df = get_aopsez_data_sql()
#rename the sql data so that we can 
readydf = Process_aop(df)
#post_AOP_data(readydf, draft=True)

In [ ]:
#VEG VIGOR#
#Use data from SEZTable until we get new lidar data
df=get_sez_data_sql()
#df= get_oldvegvig_data()
post_Veg_data(df, draft=True)


In [ ]:
#HABITAT FRAGmentation and DITCHES
#LiDAR data? this shouldn't change much 
# we can use data in the sez_habfrag_score and sez_ditches_scores tables and manually change

## Process Every Year= In-house Monitoring Data

In [ ]:
#BIOASSESSMENT#done Update EVERY YEAR after Processing BMI Data and updating SDE.Stream
#Last updated 2023 
from Bioassessment import *
# from Bioassessment import *
df = get_bioassessment_data()
#clean, process, grade data
readydf = process_grade_bioassessment(df, 2023)
#post data to sde.VECTOR.stage headcut table or csv
post_bioassessment(readydf, draft=True)


In [ ]:
#INVASIVE SPECIES#done Update EVERY YEAR after QA of data in sde.Collect (no duplicates, wrong decimal values, etc)
#Last updated with 2024 data
from Invasives import *
#Get USFS rest service data spatially joined to SEZ's
usfsdf= get_USFSinvasive_data()
#Get new invasive data from sde collect
Idf = get_combined_survey_and_invasive_data()
# *To process old data 2019-2022 uncomment the following line (this data come from GDB and not sde collect)
#Idf = get_invasive_data_gdb()
#clean, merge with external data/ format plant type names
df = merge_format_prioritize_invasive(Idf, usfsdf, 2024)
#Process Data and grade/rate assessment
#*Be sure to update the priority list in "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority Lookup.csv"/ o reven better iun the REPO
#rate_invasives is in utils if you need to update that
invasive_summary, invasive_priority_summary = process_grade_invasives(df)
#Format Data so it is ready for final QA and create csv
readydf= final_format_invasive(df, invasive_priority_summary)


Priorities: [2 'None' nan 3 4]
Draft data written to F:\GIS\GIS_DATA\Monitoring\Invasive_Species\2024 Data Processing\processedinvasivedata_2020.csv successfully.


In [ ]:
#Optional way of getting data into ARC GIS Pro
#When uploading CSV directly instead of this you need to format Year data type=SHORT
#invasives create table in scratch gdb... or just upload csv directly into ArcGIS Pro
#post data to scratch gdb for ease of appending to sez_scores_invasives table 
#df =post_invasive()

#CSV path
QAd_path = r"F:\GIS\GIS_DATA\Monitoring\Invasive_Species\2024 Data Processing\processedinvasivedata_2024.csv"
if not os.path.exists(QAd_path):
    raise FileNotFoundError(f"CSV file not found at {QAd_path}")

# Load CSV
df = pd.read_csv(QAd_path)
print(f"✅ CSV loaded: {QAd_path}")

# Output paths
gdb_path = r"F:\Research and Analysis\Workspace\Sarah\Scratch.gdb"
output_table = "invasive_temp"
output_table_path = os.path.join(gdb_path, output_table)

# Delete existing table if exists
if arcpy.Exists(output_table_path):
    arcpy.management.Delete(output_table_path)
    print(f"✅ Deleted existing table: {output_table}")

# Create empty table
arcpy.management.CreateTable(gdb_path, output_table)

# Define field types explicitly
field_types = {
    "Assessment_Unit_Name": "TEXT",
    "Year": "SHORT",
    "Invasives_Plant_Types": "TEXT",
    "Invasives_Data_Source": "TEXT",
    "Invasives_Number_of_Invasives": "DOUBLE",
    "Invasives_Rating": "TEXT",
    "Invasives_Scores": "DOUBLE",
    "Invasives_Percent_Cover": "DOUBLE"
}

# Add fields
for field_name, field_type in field_types.items():
    arcpy.management.AddField(output_table_path, field_name, field_type)
    print(f"➕ Added field: {field_name} ({field_type})")

# Make sure DataFrame only has the expected fields
df = df[list(field_types.keys())]

# Insert rows
with arcpy.da.InsertCursor(output_table_path, df.columns.tolist()) as cursor:
    for _, row in df.iterrows():
        cursor.insertRow(row.tolist())

print(f"✅ Table '{output_table}' created and populated in {gdb_path}")

In [ ]:
#HEADCUT#done#Update Every Year after QA of data in sde.Collect
#Last Updated 2024
from Headcuts import *
#Get raw headcut data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allheadcut_data()
#Get new headcut data from sde collect
df = get_combined_survey_and_headcut_data()
#clean, process, grade data
readydf = process_grade_headcut(df, 2024)
#Add Data(CSV created) to sde.sez_scores_headcut in Pro
#QA: make sure decimal points are in the right spot


Draft data written to F:\GIS\GIS_DATA\Monitoring\Stream_Headcut\Processed_Headcut_Data\processedheadcutdata_2024.csv successfully.


c:\Users\snewsome\Documents\GitHub\Monitoring\SEZ_scripts\Headcuts.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  headcutdf['Headcut_Size']=headcutdf['headcut_depth'].apply(categorize_headcut)


In [ ]:
#INCISION# Update Every Year after QA of data in sde.Collect
#Last Updated 2024
from Incision import *

#Get raw incision data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allincision_data()
#Get incision data from sde collect
df = get_combined_survey_and_incision_data()
#clean, process, grade data
readydf = process_grade_incision(df, 2024)
#Add Data(CSV created) to sde.sez_scores_incision in Pro
#QA:get rid of Null values until you add this to the code for QA SEZ Type, Check calculated vs incision ratio to make sure caluclations make sense

Draft data written to F:\GIS\GIS_DATA\Monitoring\Channel_Incision\Processed_Incision_Data\processedincisiondata_2024.csv successfully.


In [12]:
#EROSION# Update Every Year after QA of data in sde.Collect
#Last updated 2024
from Erosion import *

#Get raw erosion data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
df= get_erosion_data()

#clean, process, grade data
readydf = process_grade_erosion(df, 2024)
#Add Data(CSV created) to sde.sez_scores_incision in Pro
#QA: make sure decimal points are in the right spot, check for null values, check for duplicates, check for correct SEZ type

Data written to F:\GIS\GIS_DATA\Monitoring\Stream_Erosion\Processed_Erosion_Data\processederosiondata_2024.csv successfully.


c:\Users\snewsome\Documents\GitHub\Monitoring\SEZ_scripts\Erosion.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bank_multiplier'] = df['Bank_Type'].map({'Both Banks': 2, 'One Bank': 1, 'No Bank': 0})
c:\Users\snewsome\Documents\GitHub\Monitoring\SEZ_scripts\Erosion.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['eroded_banks_per_row'] = df['Shape_Length'] * df['bank_multiplier']


# Part 2: Staging Tables to Final Table

## 

In [ ]:

from FormatStagingTables import *
# Staging to final
dfbanks = get_fs_data(bank_stability_url)
dfbiotic = get_fs_data(biotic_integrity_url)
dfconifer = get_fs_data(conifer_url)
dfditch = get_fs_data(ditches_url)
dfinvasive = get_fs_data(invasives_url)
dfhabitat = get_fs_data(Hab_Frag_url)
dfvegetation = get_fs_data(vegetation_url)
dfincision = get_fs_data(incision_url)
dfheadcuts = get_fs_data(headcuts_url)
dfAOP = get_fs_data(AOP_url)
dfSEZ = get_fs_data_spatial(SEZ_url)
dfheadcuts.drop(columns=['small', 'medium', 'large'], inplace=True)
#Format Staging Tables
#format biotic data average if two streams in one assessment unit
averaged_biotic_df=average_biotic_scores(dfbiotic)

#Same for meadow(large polygon) and riverine(small polygon) data drop these columns because not needed in final merge, will assign SEZ ID later
columns_to_drop = ['Year', 'SEZ_ID', 'GlobalID', 'last_edited_user', 'created_date', 'OBJECTID', 'created_user', 'last_edited_date']

#Create Dictionary of Dataframes to adjust year to be in datashource column and not its own column
yeartodatasource = {
    'dfbanks': dfbanks,
    'dfheadcuts': dfheadcuts,
    'dfincision': dfincision,
    'dfinvasive': dfinvasive
}

# Iterate over each DataFrame in meadowdata
for name, df in yeartodatasource.items():
    # Iterate over columns in the DataFrame
    for col in df.columns:
        # Check if the column name contains 'Data'
        if 'Data_' in col:
            # Add Year to the column if it contains 'Data'
            df[col] = df[col].astype(str) + ', ' + df['Year'].astype(str)

#Name dataframes so we can reference later
largepolygondata= {'dfbanks': dfbanks, 
                'dfaveraged_biotic': averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}


#Staging Tables Riverine/ small polygons
smallpolygondata = {'dfbanks': dfbanks, 
                'dfaveraged_biotic': averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}


# Process large polygon (meadow) and small polygon (riverine) data -
#Get most recent scores for each indicator
# Clean Data in each dataframe, assign SEZ ID to each assessment unit, add year to datasource column for formatting purposes
processed_largepolygon_data = process_data(largepolygondata, lookup_dict, columns_to_drop)
processed_smallpolygon_data = process_data(smallpolygondata, lookup_riverine, columns_to_drop)


# Merge large polygon and small polygon data
# Function to merge all DataFrames on multiple keys)
# # #moved to formatstagingtables.py
def merge_dataframes(data_dict, keys):
    return reduce(lambda left, right: pd.merge(left, right, on=keys, how='inner'), data_dict.values())


# # Merge small polygon DataFrames
smallpolygon_df = merge_dataframes(processed_smallpolygon_data, ['SEZ_ID', 'Assessment_Unit_Name' ])

# Merge large polygon DataFrames so all indicators that have data
largepolygon_df = merge_dataframes(processed_largepolygon_data, ['SEZ_ID', 'Assessment_Unit_Name' ])


# Append smallpolygon_df to largepolygon_df to have a complete list of all meadows and all sez ids
final_combined_df = pd.concat([largepolygon_df, smallpolygon_df], ignore_index=True)

# Print the final combined DataFrame to check
#print("Final Combined DataFrame:")
print(final_combined_df)


# # Join SEZinfo to combined df to get a dataframe with more info about each Assessment Unit
# #Prep SEZ Baseline Data for assessment unit...will need to rethink if acreage changes.. or just manually change in sde
keep_columns = ['SHAPE', 'Assessment_Unit_Name', 'Threshold_Year', 'SEZ_ID', 'Feature_Type', 'SEZ_Type', 'Ownership_Primary', 'Ownership_Secondary', 'Ownership_Secondary_2', 'Ownership_Secondary_3', 'Acres', 'Comments']
# #dfSEZ is assessment unit information from SDE
dfSEZ['SEZ_ID']= dfSEZ['SEZ_ID'].astype(int)
dfSEZinfo= dfSEZ.loc[:,keep_columns].copy()

#filter dfSEZinfo for threshol dyear ='2019'
dfSEZinfo = dfSEZinfo[dfSEZinfo['Threshold_Year'] == '2019']

# # Join SEZinfo to the combined_df using SEZ_ID
final_df = pd.merge(final_combined_df, dfSEZinfo, on=['SEZ_ID', 'Assessment_Unit_Name'], how='right')
#-----------------------------------
# Define Year Data is processed
#-----------------------------------
# #Assign Threshold Calculations which is the Threshold Year--> is just the most recent data within the past 4 years
final_df['Threshold_Year'] = 'test'

# Step 1: Get the most recent year in the dataset
#most_recent_year = final_df['Year'].max()

# Step 2: Define the Threshold_Year based on the most recent year
# If the data's year is within the last 4 years from the most recent year, it's considered valid
#final_df['Threshold_Year'] = final_df['Year'].apply(
#    lambda x: most_recent_year if most_recent_year - x <= 4 else None
#)

# Optional: Print to verify
#print(f"Most Recent Year: {most_recent_year}")

print(final_df)

In [ ]:
# remove values for indicators based on sez type
#Use old data from most recent year evaluation to fill in gaps. (data from 2019 that was updated with external data such as simon sedimentation ratings)

#  Use new data and fill in nulls with old data

In [ ]:

#Bring in Last year threshold scores for any indicators thataren't in our raw data- some bank stability came from our stream surveys but aren't in the data
#Fill in any indicators that have missing data with data from 2019

# Rename fields based on field mappings
dfSEZprep = dfSEZ.rename(columns=field_mapping)

# Ensure 'Threshold Year' is properly filtered
dfSEZprep = dfSEZprep[dfSEZprep['Threshold Year'] == '2019']

# Filter out columns in dfSEZprep that are not in final_df
dfSEZprep = dfSEZprep[[col for col in final_df.columns if col in dfSEZprep.columns]]

# Set SEZ_ID as index for both DataFrames
final_df.set_index('SEZ_ID', inplace=True)
dfSEZprep.set_index('SEZ_ID', inplace=True)

# Ensure dfSEZprep has no duplicate SEZ_IDs
dfSEZprep = dfSEZprep[~dfSEZprep.index.duplicated(keep='last')]

# Align dfSEZprep index with final_df before filling missing values
final_df = final_df.fillna(dfSEZprep.reindex(final_df.index))

# Reset the index to restore SEZ_ID as a column
final_df.reset_index(inplace=True)

print(dfSEZprep)
print(final_df)


## QA tool

In [ ]:
#-------------
#QAOnly
#------------
# #RUN FOR QA so you can see the resulting dataframe/scores/formatting etc.
#CSV will be posteed to the SEZ_scripts folder in file explorer
# List of columns to export (excluding 'geometry' or 'Shape')
columns_to_export = [col for col in final_df.columns if col != 'SHAPE']
final_df[columns_to_export].to_csv('ready_df.csv', index=False)

# Calcluate SEZ Quality based on SEZ Type

In [ ]:
from SEZScores import *
#Calculate Score based on SEZ_Type  #Future- make it so any indicator that isn't in the lists below for SEZ Type says NA
#Use SEZ_Type to select only needed indicators for SEZ Type
# Define the score columns needed for each SEZ Type
# Function to get the score columns based on SEZ_Type


# Apply the appropriate score columns based on SEZ_Type
final_df['Score_Columns'] = final_df['SEZ_Type'].apply(get_score_columns)


# Apply the score calculation to each row
final_df[['Final_Total_Points', 'Final_Points_Possible']] = final_df.apply(calculate_scores, axis=1)

# Calculate the final percent
final_df['Final_Percent'] = final_df['Final_Total_Points'] / final_df['Final_Points_Possible']

# Calculate the final rating and score
final_df['Final_Rating'] = final_df['Final_Percent'].apply(rate_SEZ)
final_df['Final_Score'] = final_df['Final_Rating'].apply(score_indicator)

# Drop the temporary 'Score_Columns' column
final_df = final_df.drop(columns=['Score_Columns'])
print(final_df)



In [ ]:
#Final set up


#Field Mapping
field_mapping = {
    'Assessment_Unit_Name': 'Assessment_Unit_Name',
    'SHAPE': 'SHAPE',
    'Threshold_Year': 'Threshold Year',
    'Comments': 'Comments',
    'SEZ_Type': 'SEZ_Type',
    'Final_Rating': 'Final_Rating',
    'Final_Percent': 'Final_Percent',
    'SEZ_ID': 'SEZ_ID',
    'Final_Points_Possible': 'Final_Points_Possible',
    'Final_Total_Points': 'Final_Total_Points',
    'Acres': 'Acres',
    'AOP_BarriersPerMile':'AquaticOrganismPassage_Barriers',
    'AOP_NumberofBarriers': 'AquaticOrganismPassage_NumberOf',
    'AOP_Score': 'AquaticOrganismPassage_Score',
    'AOP_Rating': 'AquaticOrganismPassage_Rating',
    'AOP_StreamMiles': 'AquaticOrganismPassage_StreamMiles',
    'AOP_DataSource': 'AquaticOrganismPassage_DataSour',
    'Bank_Stability_Data_Source': 'Bank_Stability_Data_Source',
    'Bank_Stability_Percent_Unstable': 'Bank_Stability_Percent_Unstable',
    'Bank_Stability_Rating': 'Bank_Stability_Rating',
    'Bank_Stability_Score': 'Bank_Stability_Score',
    'Biotic_Integrity_Rating': 'Biotic_Integrity_Rating',
    'Biotic_Integrity_CSCI': 'Biotic_Integrity_CSCI',
    'Biotic_Integrity_Data_Source': 'Biotic_Integrity_Data_Source',
    'Biotic_Integrity_Score': 'Biotic_Integrity_Score',
    'Comments': 'Comments',
    'Conifer_Percent_Encroached': 'Conifer_Encroachment_Percent_En',
    'Conifer_Encroachment_Data_Sourc': 'Conifer_Encroachment_Data_Sourc',
    'Conifer_Encroachment_Rating': 'Conifer_Encroachment_Rating',
    'Conifer_Encroachment_Score': 'Conifer_Encroachment_Score',
    'ConiferEncroachment_Comments': 'Conifer_Encroachment_Comments',
    'Ditches_Data_Source': 'Ditches_Data_Source',
    'Ditches_Length': 'Ditches_Length',
    'Ditches_Meadow_Length': 'Ditches_Meadow_Length',
    'Ditches_Percent': 'Ditches_Percent',
    'Ditches_Rating': 'Ditches_Rating',
    'Ditches_Score': 'Ditches_Score',
    'Feature_Type': 'Feature_Type',
    'Habitat_Frag_Data_Source': 'Habitat_Fragmentation_Data_Sour',
    'Habitat_Frag_Impervious_Acres': 'Habitat_Fragmentation_Imperviou',
    'Habitat_Frag_Percent_Impervious': 'Habitat_Fragmentation_Percent_I',
    'Habitat_Frag_Rating': 'Habitat_Fragmentation_Rating',
    'Habitat_Frag_Score': 'Habitat_Fragmentation_Score',
    'Headcuts_Data_Source': 'Headcuts_Data_Source',
    'Number_of_Headcuts': 'Headcuts_Number_of_Headcuts',
    'Headcuts_Rating': 'Headcuts_Rating',
    'Headcuts_Score': 'Headcuts_Score',
    'Incision_Data_Source': 'Incision_Data_Source',
    'Incision_Rating': 'Incision_Rating',
    'Incision_Score': 'Incision_Score',
    'Incision_Ratio': 'Incision_Ratio',
    'Invasive_Percent_Cover': 'Invasive_Percent_Cover',
    'Invasives_Rating': 'Invasive_Rating',
    'Invasives_Data_Source': 'Invasives_Data_Source',
    'Invasives_Number_of_Invasives': 'Invasives_Number_of_Invasives',
    'Invasives_Plant_Types': 'Invasives_Plant_Types',
    'Invasives_Scores': 'Invasives_Scores',
    'NDVI_ID': 'NDVI_ID',
    'Ownership_Primary': 'Ownership_Primary',
    'Ownership_Secondary': 'Ownership_Secondary',
    'Ownership_Secondary_2': 'Ownership_Secondary_2',
    'Ownership_Secondary_3': 'Ownership_Secondary_3',
    'VegetationVigor_DataSource': 'VegetationVigor_DataSource',
    'VegetationVigor_Rating': 'VegetationVigor_Rating',
    'VegetationVigor_Raw': 'VegetationVigor_Raw',
    'VegetationVigor_Score': 'VegetationVigor_Score'
}
# Rename fields based on field mappings
SEZscores_readydf = final_df.rename(columns=field_mapping).drop(columns=[col for col in final_df.columns if col not in field_mapping])


print(SEZscores_readydf)
#----------------------------------------------------
#Post results to CSV in gis/projects/Researchanalysis/SEZ for further QA
#----------------------------------------------------
 
#export all columns but shape
columns_to_export = [col for col in SEZscores_readydf.columns if col != 'SHAPE']
#Store csv on F drive for QA/add comments manually on F drive and to change up comments later based on SEZ's that scores changed
#final_results = "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Threshold24_SEZScores.csv"

# Write to excel
#SEZscores_readydf[columns_to_export].to_excel(final_results, index=False)


# Convert DataFrame to a list of dictionaries
#data = readydf.to_dict(orient='records')

# Get the field names from the field mapping
#field_names = list(readydf.columns)

# Append data to existing table
#with arcpy.da.InsertCursor(allsezscores, field_names) as cursor:
 #   for row in data:
  #      cursor.insertRow([row[field] for field in field_names])

#print("Data appended to table successfully.")

## QA Tools

In [ ]:
#This creates a csv of score changes and all information in row 

#Before Final comparison do more QA
    # Check to make sure there are 641 ASsessment Units
    #Check Final Points and make sure there are no really low numbers could be missing data
    #Check all staging tables for completeness- There are ratings A-D, check on null data for that SEZ
    #other QA methods can be added to this list

#Check to see which SEZ Scores Changed
Threshold23sezdata= SEZscores_readydf
Threshold19sezdata = dfSEZ

# Merge the datasets on Assessment_Unit_Name and SEZ_ID
merged_df = pd.merge(
    Threshold23sezdata[['Assessment_Unit_Name', 'SEZ_ID', 'Final_Rating', 'SEZ_Type']],
    Threshold19sezdata[['Assessment_Unit_Name', 'SEZ_ID', 'Final_Rating', 'SEZ_Type']],
    on=['Assessment_Unit_Name', 'SEZ_ID'],
    suffixes=('_2023', '_2019')
)

# Identify Assessment Unit Names with a change in Final Rating
changed_scores_df = merged_df[merged_df['Final_Rating_2023'] != merged_df['Final_Rating_2019']]

# List of Assessment Unit Names with a change in score
changed_assessment_units = changed_scores_df[['Assessment_Unit_Name', 'SEZ_ID', 'Final_Rating_2023', 'Final_Rating_2019']]

# Output the DataFrame with changed scores
print(changed_scores_df)

# Output the list of Assessment Unit Names with a change in score
print(changed_assessment_units)

#Post results to CSV in gis/projects/Researchanalysis/SEZ for further QA
columns_to_export = [col for col in changed_assessment_units.columns if col != 'SHAPE']
#Store csv on F drive for QA/add comments manually on F drive and to change up comments later based on SEZ's that scores changed
#Changed_Scores_List = "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Threshold24_SEZScoreChanges.csv"
# Assuming ready_df is the DataFrame you want to write to CSV
#changed_scores_df.to_csv(Changed_Scores_List, index=False)

In [ ]:
#Otion for different CSV Look at only changed scores so we can update comments and make sure data is good, need to run this to get long format csv
# List of all columns with 'Rating' in their name for both datasets
rating_columns_2023 = [col for col in Threshold23sezdata.columns if 'Rating' in col]
rating_columns_2019 = [col for col in Threshold19sezdata.columns if 'Rating' in col]

# Ensure both DataFrames have the same Assessment_Unit_Name and SEZ_ID columns
Threshold23sezdata = Threshold23sezdata[['Assessment_Unit_Name', 'SEZ_ID'] + rating_columns_2023]
Threshold19sezdata = Threshold19sezdata[['Assessment_Unit_Name', 'SEZ_ID'] + rating_columns_2019]

# Merge the datasets on Assessment_Unit_Name and SEZ_ID, for units that have changed scores
final_merged_df = pd.merge(
    Threshold23sezdata,
    Threshold19sezdata,
    on=['Assessment_Unit_Name', 'SEZ_ID'],
    suffixes=('_2023', '_2019')
)

# Filter the final merged DataFrame to only include units that changed scores
final_changed_scores_df = final_merged_df[final_merged_df['Assessment_Unit_Name'].isin(changed_assessment_units['Assessment_Unit_Name'])]

# Print the final DataFrame with all rating columns and changed scores
print(final_changed_scores_df)




# Save the final DataFrame to a CSV file
#final_changed_scores_df.to_csv("F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Threshold24_SEZScoreChanges_indicators.csv", index=False)


In [ ]:
#BEST QA csv.. shows indicators scores and final score changes by year of threshold
#futureadd in final points and points possible and comments
# Used changed changed_assessment_units to get Rating scores from df's 
#Create CSV with assessment units that have changed , all indicator scores and final rating by year and SEZID

# List of all columns with 'Rating' in their name for both datasets
rating_columns_2023 = [col for col in Threshold23sezdata.columns if 'Rating' in col]
rating_columns_2019 = [col for col in Threshold19sezdata.columns if 'Rating' in col]

# Ensure both DataFrames have the same Assessment_Unit_Name and SEZ_ID columns
Threshold23sezdata = Threshold23sezdata[['Assessment_Unit_Name', 'SEZ_ID'] + rating_columns_2023]
Threshold19sezdata = Threshold19sezdata[['Assessment_Unit_Name', 'SEZ_ID'] + rating_columns_2019]

# Merge changed_assessment_units with Threshold23sezdata and Threshold19sezdata to get ratings for changed units
merged_23 = pd.merge(changed_assessment_units, Threshold23sezdata, on=['Assessment_Unit_Name', 'SEZ_ID'])
merged_19 = pd.merge(changed_assessment_units, Threshold19sezdata, on=['Assessment_Unit_Name', 'SEZ_ID'])

# Extract columns with 'Rating' for both datasets
rating_columns_2023 = [col for col in merged_23.columns if 'Rating' in col]
rating_columns_2019 = [col for col in merged_19.columns if 'Rating' in col]

# Filtered DataFrames with only relevant columns
filtered_23 = merged_23[['Assessment_Unit_Name', 'SEZ_ID'] + rating_columns_2023]
filtered_19 = merged_19[['Assessment_Unit_Name', 'SEZ_ID'] + rating_columns_2019]

# Add a 'Year' column to each DataFrame
filtered_23['Year'] = '2023'
filtered_19['Year'] = '2019'

# Concatenate filtered_23 and filtered_19 DataFrames
allratings = pd.concat([filtered_23, filtered_19], ignore_index=True)


#Function to map SEZ_Type based on Assessment_Unit_Name
def get_sez_type(row):
    return lookup_all.get(row['SEZ_ID'], {}).get('SEZ_Type', None)

# Apply the function to add SEZ_Type column
allratings['SEZ_Type'] = allratings.apply(get_sez_type, axis=1)

# Apply the function to add SEZ_Type column
allratings['SEZ_Type'] = allratings.apply(get_sez_type, axis=1)

# Drop the specified columns from merged_df
columns_to_drop = ['Final_Rating_2019', 'Final_Rating_2023']
allratings.drop(columns=columns_to_drop, inplace=True)

# Save the final long format DataFrame to a CSV file
#allratings.to_csv("F:\\GIS\\PROJECTS\\ResearchAnalysis\\SEZ\\allchangedscores_longformat.csv", index=False)

allratings.to_csv("C:\\Users\\snewsome\\Documents\\SEZallchangedscores_longformat.csv", index=False)

# Note to SELF - when group editing csv and fixing comments for changed scores this must be in an excel and is easiest if put online into google office sheets. 

# I would like to code the comments eventually. For example if the Rating is a C then the comment should start with "This meadow is moderately degraded yet stable since the scores have not changed in blank years. Describe the indicators that are causing the score to be what it is"